# Integrated YOLO + Drift Visualization (with Coordinates)

This notebook integrates the statistical drift detection results with live YOLOv8 object detection.
It differs from the previous analysis by explicitly merging `trajectory_data` (coordinates) with `behavior_features` (anomalies) to draw bounding boxes around **drifting** individuals.

In [16]:
from ultralytics import YOLO
import cv2
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

In [17]:
# --- 1. Load and Merge Data ---
print("Loading Data...")

traj_path = "D:\\Projects\\Human-Behavior-Drift-Detection-System\\data\\trajectory_data\\trajectory_data_3.csv"
feat_path = "D:\\Projects\\Human-Behavior-Drift-Detection-System\\data\\Processed\\behavior_features_3.csv"

if not os.path.exists(traj_path) or not os.path.exists(feat_path):
    # Fallback for relative paths
    traj_path = "../data/trajectory_data/trajectory_data_3.csv"
    feat_path = "../data/Processed/behavior_features_3.csv"

# Load Trajectory (Has Coordinates x, y)
df_traj = pd.read_csv(traj_path)
# Sort by Person then Time to match Feature Extraction order assumption
df_traj = df_traj.sort_values(["person_id", "time"]).reset_index(drop=True)

# Load Features (Has Behavior Metrics)
df_feat = pd.read_csv(feat_path)
# df_feat IS implied to be in the same order as sorted trajectory

# Check lengths
if len(df_traj) != len(df_feat):
    print(f"Warning: Length mismatch! Traj: {len(df_traj)}, Feat: {len(df_feat)}")
    # Attempt to align by index if possible, or truncate
    min_len = min(len(df_traj), len(df_feat))
    df_traj = df_traj.iloc[:min_len]
    df_feat = df_feat.iloc[:min_len]

# Concatenate
df = pd.concat([df_traj, df_feat.drop(columns=["time"], errors="ignore")], axis=1)

print(f"Merged DataFrame Shape: {df.shape}")
print(df.head())

Loading Data...
Merged DataFrame Shape: (8562, 9)
   person_id  time           x           y     speed  direction  density  \
0          2     0  544.035461  307.354980       NaN        NaN       12   
1          2     1  543.740845  307.731567  0.478139   2.234672       10   
2          2     2  543.031494  308.170837  0.834348   2.587134       11   
3          2     3  543.017090  308.402496  0.232106   1.632895       11   
4          2     4  542.682434  308.570984  0.374677   2.675177       13   

   direction_entropy  avg_neighbor_dist  
0          -0.000000          24.209144  
1           1.609438          28.804466  
2           1.418484          29.201258  
3           1.846220          29.390634  
4           1.263654          23.934665  


In [18]:
# --- 2. Re-Run Drift Detection ---
# We re-run Isolation Forest here to ensure we have 'anomaly' labels for valid rows

features = ["speed", "direction", "density", "direction_entropy", "avg_neighbor_dist"]
data_clean = df.dropna(subset=features).copy()

print(f"Training Isolation Forest on {len(data_clean)} samples...")
scaler = StandardScaler()
X_scaled = scaler.fit_transform(data_clean[features])

iso_forest = IsolationForest(n_estimators=100, contamination=0.05, random_state=42)
data_clean["anomaly_score"] = iso_forest.fit_predict(X_scaled)
data_clean["anomaly"] = data_clean["anomaly_score"].apply(lambda x: "drift" if x == -1 else "normal")

print("Drift Detection Complete.")
print(data_clean["anomaly"].value_counts())

Training Isolation Forest on 8466 samples...
Drift Detection Complete.
anomaly
normal    8042
drift      424
Name: count, dtype: int64


In [19]:
# --- 3. Initialize YOLO ---
print("Loading YOLOv8n...")
model = YOLO("yolov8n.pt")
print("Model Loaded.")

Loading YOLOv8n...
Model Loaded.


In [20]:
# --- 4. Visualization Loop ---

video_path = "D:\\Projects\\Human-Behavior-Drift-Detection-System\\data\\raw\\Arxiepiskopi_flock.avi"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    raise FileNotFoundError("Error opening video stream or file")

# Create a lookup map: Time -> DataFrame of objects in that frame
frame_map = dict(tuple(data_clean.groupby("time")))

frame_id = 0
print("Starting Video... Press 'q' to exit.")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO once per frame
    results = model(frame, verbose=False)
    yolo_boxes = []  # [(x1,y1,x2,y2,cx,cy,cls,conf)]
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            cx_box = int((x1 + x2) / 2)
            cy_box = int((y1 + y2) / 2)
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            yolo_boxes.append((x1, y1, x2, y2, cx_box, cy_box, cls_id, conf))

    # --- A. Draw Drift Boxes (Red/Green) ---
    if frame_id in frame_map:
        frame_data = frame_map[frame_id]

        drift_count = (frame_data["anomaly"] == "drift").sum()
        status_color = (0, 0, 255) if drift_count > 0 else (0, 255, 0)

        cv2.putText(frame, f"Frame: {frame_id} | Drift: {drift_count}", (30, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, status_color, 2)

        # Draw drift/normal per person
        for _, row in frame_data.iterrows():
            cx, cy = int(row['x']), int(row['y'])
            is_drift = row['anomaly'] == 'drift'
            color = (0, 0, 255) if is_drift else (0, 255, 0)

            # Draw point + ID
            cv2.circle(frame, (cx, cy), 5, color, -1)
            cv2.putText(frame, str(int(row['person_id'])), (cx+5, cy-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

            if is_drift:
                # Match nearest YOLO box and draw it in red
                nearest = None
                min_dist = 1e9
                for (x1, y1, x2, y2, bx, by, cls_id, conf) in yolo_boxes:
                    dist = (bx - cx) ** 2 + (by - cy) ** 2
                    if dist < min_dist:
                        min_dist = dist
                        nearest = (x1, y1, x2, y2)
                if nearest is not None and min_dist <= 50 * 50:
                    x1, y1, x2, y2 = nearest
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                else:
                    # fallback: highlight drift with a larger circle
                    cv2.circle(frame, (cx, cy), 15, (0, 0, 255), 2)

    # --- B. Draw YOLO Boxes (Blue) ---
    for (x1, y1, x2, y2, _, _, cls_id, conf) in yolo_boxes:
        cls_name = model.names[cls_id]
        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(frame, f"{cls_name} {conf:.2f}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    cv2.imshow("Drift (Red) vs YOLO (Blue)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_id += 1

cap.release()
cv2.destroyAllWindows()

Starting Video... Press 'q' to exit.
